In [3]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import re
import nltk

In [4]:
df = pd.read_excel("Product Matching Dataset.xlsx" , sheet_name="Dataset")
df.head(n=10)

,sku,marketplace_product_name_ar,seller_item_name,price
0,1322,استوهالت 40 مجم 14 كبسول,ESTOHALT 40 MG 14 CAP,56.5
1,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 مجم 14 ك,56.5
2,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 مجم 14 ك,56.5
3,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 مجم 14 ك,56.5
4,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 مجم 14 ك,56.5
5,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 مجم 14 ك,56.5
6,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 مجم 14 ك,56.5
7,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 مجم 1 شريط * 14 كبسولة,56.5
8,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 مجم,56.5
9,1322,استوهالت 40 مجم 14 كبسول,استوهالت 40 كبسول س ج,56.5


In [5]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words("arabic"))
text_column = df["seller_item_name"].astype(str)  # Ensure text format

In [6]:
texts = []
for text in text_column:
    # Remove stopwords
    text = " ".join(word for word in text.split() if word not in stop_words)
    # Remove punctuation & special characters (Keep Arabic, English, and Numbers)
    text = re.sub(r"[^\w\s\u0600-\u06FF]", " ", text)
    # Remove Arabic diacritics (Tashkeel)
    text = re.sub(r"[\u064B-\u065F]", "", text)

    text = re.sub(r'[إأآ]', 'ا', text)
    text = re.sub(r'ى', 'ي', text)
    text = re.sub(r'ة', 'ه ', text)
    text = re.sub(r'ؤ', 'و', text)
    text = re.sub(r'ئ', 'ي', text)
    text = re.sub(r'ــ', '', text)
    text = re.sub(r"(\d+)", r" \1 ", text)
    text = re.sub(r"\b[\u0600-\u06FF]\b", "", text)
    text = re.sub(r'([\u0600-\u06FF])\1{2,}', r'\1\1', text)
    text = re.sub(r'(.)\1+', r'\1', text) # remove repetitions
    # Remove short/incomplete words (single characters or partial words)
    text = re.sub(r'\b\w\b', '', text)  # Removes standalone single characters
    text = re.sub(r'\b(?:سعر جديد|س جديد|س جدي|س ج|ركز)\b', '', text)# Remove specific unwanted phrases
    text = re.sub( r'مرهم|اكريم', 'كريم', text)
    text = re.sub( r'قرص|اقراص|كبسوله', 'كبسول', text)
    text = re.sub( r'اقراص|شريط|شرائط|شريطين', 'قرص', text)
    text = re.sub( r'امبولات|امبوله|حقن', 'امبول', text)
    text = re.sub( r'لبوس|لبوس اطفال|لبوس اطفال فاركو', 'اقماع للاطفال', text)
    text = re.sub(r"\s+", " ", text).strip()
    # Append cleaned text
    texts.append(text)

In [7]:
texts[14260:14270 ]

['فورتيموكس 10 ملي نقط',
 'فورتيموكس قطره عين 10',
 'فورتيموكس قطره عين 10',
 'فورتيموكس مجم مل قطره 10 مل',
 'فورتي موكس نقط جديد',
 'فورتيموكس قطره',
 'فورتيموكس قطره',
 'فورتيمكس نقط',
 'فورتيمكس نقط',
 'فورتيموكس قطره']

In [8]:
x=texts
y=df["sku"]
vectorize = TfidfVectorizer(max_features = 500)
x_tfidf = vectorize.fit_transform(x)

X_train, X_test, y_train, y_test = train_test_split(x_tfidf, y, test_size=0.3, random_state=42)

# Train Logistic Regression Model
model = LogisticRegression()
model.fit(X_train, y_train)

# Predict on Test Data
y_pred = model.predict(X_test)

# Evaluate Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.4f}")

Model Accuracy: 0.8627


In [10]:
from symspellpy import SymSpell

# Assuming the column containing product names is named 'Product_Name'
product_column = "marketplace_product_name_ar"  # Change this to the actual column name

# Create a dictionary for SymSpell (word -> frequency)
word_freq_dict = {}

# Count occurrences of each product name
for product in df[product_column].dropna():
    normalized_product = str(product).strip()  # Ensure text format and remove spaces
    word_freq_dict[normalized_product] = word_freq_dict.get(normalized_product, 0) + 1

# Save dictionary to a text file
dict_file = "product_dictionary.txt"
with open(dict_file, "w", encoding="utf-8") as f:
    for word, freq in word_freq_dict.items():
        f.write(f"{word} {freq}\n")

# Initialize SymSpell
sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)

In [12]:
from difflib import SequenceMatcher

sym_spell.load_dictionary(dict_file, term_index=0, count_index=1, encoding="utf-8")

# Test with product names from the dataset
# Specify the correct column containing product names
product_column = "marketplace_product_name_ar"  # Change if needed
product_names = df[product_column].dropna().unique()  # Remove NaN and get unique names

# Function to get the best match based on similarity
def get_best_match(input_name, product_names):
    best_match = None
    highest_similarity = 0

    for product_name in product_names:
        similarity = SequenceMatcher(None, input_name, product_name).ratio()

        if similarity > highest_similarity:
            highest_similarity = similarity
            best_match = product_name

    return best_match, highest_similarity

# Test with product names (misspelled inputs)
test_names = [
    "استوهلت 40 مجم كسبول",  # Misspelled كبسول
    "ريتاس 5مجم س ج 57",  # Misspelled 40 مجم
    "استوهالت 40 جم كسبول",  # Misspelled جم كبسول
    "اماريل 3مج30ق س ج/سانوفي",  # Different form of كبسول
]

print("\n=== Product Name Correction with Similarity Checking ===")
for test_name in test_names:
    best_match, similarity = get_best_match(test_name, product_names)
    
    if best_match:
        print(f"Input: {test_name} -> Suggested: {best_match} (Similarity: {similarity:.2f})")
    else:
        print(f"Input: {test_name} -> No suggestions found.")


=== Product Name Correction with Similarity Checking ===
Input: استوهلت 40 مجم كسبول -> Suggested: استوهالت 40 مجم 14 كبسول (Similarity: 0.86)
Input: ريتاس 5مجم س ج 57 -> Suggested: تريتاس 5 مجم 14 قرص (Similarity: 0.67)
Input: استوهالت 40 جم كسبول -> Suggested: استوهالت 40 مجم 14 كبسول (Similarity: 0.86)
Input: اماريل 3مج30ق س ج/سانوفي -> Suggested: اماريل 3 مجم 30 قرص (Similarity: 0.60)
